In [3]:
from langchain.llms import GooglePalm
import os
from dotenv import load_dotenv
load_dotenv()

google_api_key = os.environ["GOOGLE_API_KEY"]


In [11]:
google_llm = GooglePalm(google_api_key=google_api_key, temperature=0.2)

ans=google_llm('''Question:What is the price of 3 nike white tshirt? Answer:1800rs prompt:your are a inventory manager Walmart store ,you got the question from employee. 
               response professionally with the answer to employee ''')

print(ans)

Thank you for your question. The price of 3 Nike white t-shirts is 1800rs.


In [4]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "91"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price_per_unit * stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "22292"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price_per_unit * stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "16725.4"} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price_per_unit * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "17462"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
     },
    {'Question': "How much sales amount will be generated if we sell all large size t-shirts today in Nike brand after discounts?",
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price_per_unit * stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike' and size='L'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
    }
]

to_vectorize = [" ".join(example.values()) for example in few_shots]

to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 91",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price_per_unit * stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 22292",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price_per_unit * stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 16725.4",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price_per_unit * stock_quantity) FROM

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings= HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

vs = FAISS.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [17]:

dir(vs)

# vs.similarity_search_with_score_by_vector("hello")
dir(embeddings)
# embeddings.parse_raw("hello")


{'title': 'HuggingFaceEmbeddings',
 'description': 'HuggingFace sentence_transformers embedding models.\n\nTo use, you should have the ``sentence_transformers`` python package installed.\n\nExample:\n    .. code-block:: python\n\n        from langchain.embeddings import HuggingFaceEmbeddings\n\n        model_name = "sentence-transformers/all-mpnet-base-v2"\n        model_kwargs = {\'device\': \'cpu\'}\n        encode_kwargs = {\'normalize_embeddings\': False}\n        hf = HuggingFaceEmbeddings(\n            model_name=model_name,\n            model_kwargs=model_kwargs,\n            encode_kwargs=encode_kwargs\n        )',
 'type': 'object',
 'properties': {'client': {'title': 'Client'},
  'model_name': {'title': 'Model Name',
   'default': 'sentence-transformers/all-mpnet-base-v2',
   'type': 'string'},
  'cache_folder': {'title': 'Cache Folder', 'type': 'string'},
  'model_kwargs': {'title': 'Model Kwargs', 'type': 'object'},
  'encode_kwargs': {'title': 'Encode Kwargs', 'type': 'obj